In [1]:
import pandas as pd

#Carga de datos
filepath_dict = {'yelp':   'yelp_labelledes.csv',
                 'imdb':   'imdb_labelledes.csv',
                 'amazon':   'amazon_cells_labelledes.csv'}

df_list = []
for source, filepath in filepath_dict.items():
    df = pd.read_csv(filepath, names=['sentence', 'label'], sep='\t')
    df['source'] = source
    df_list.append(df)

df = pd.concat(df_list)
print(df.iloc[0])

sentence    Wow ... Me encantó este lugar.
label                                    1
source                                yelp
Name: 0, dtype: object


In [2]:
df.head(10)

,sentence,label,source
0,Wow ... Me encantó este lugar.,1,yelp
1,La corteza no es buena.,0,yelp
2,No sabroso y la textura era simplemente desagr...,0,yelp
3,Me detuve durante las vacaciones de fines de m...,1,yelp
4,"La selección en el menú era genial, al igual q...",1,yelp
5,Ahora me estoy enojando y quiero mi maldito pho.,0,yelp
6,Honestamente no sabía tan fresco.),0,yelp
7,Las papas eran como caucho y se notaba que se ...,0,yelp
8,Las patatas fritas también eran geniales.,1,yelp
9,Un gran toque.,1,yelp


In [3]:
from sklearn.feature_extraction.text import CountVectorizer

#Ejemplo de vectorización
sentences = ['A Juan le gusta el chocolate', 'Juan odia el chocolate']
vectorizer = CountVectorizer(min_df=0, lowercase=False)
vectorizer.fit(sentences)
vectorizer.vocabulary_


{'Juan': 0, 'le': 4, 'gusta': 3, 'el': 2, 'chocolate': 1, 'odia': 5}

In [4]:
vectorizer.transform(sentences).toarray()

array([[1, 1, 1, 1, 1, 0],
       [1, 1, 1, 0, 0, 1]])

In [5]:
from sklearn.model_selection import train_test_split
#Creamos un clasificador simple con los datos relativos a "Yelp"
df_yelp = df[df['source'] == 'yelp']
#Preparamos los datos para el entrenamiento del modelo
#Obtenemos las sentencias a clasificar y las etiquetas relativas a estas
sentences = df_yelp['sentence'].values
y = df_yelp['label'].values

sentences_train, sentences_test, y_train, y_test = train_test_split(
   sentences, y, test_size=0.25, random_state=1000)

In [6]:
from sklearn.feature_extraction.text import CountVectorizer
#Vectorizamos los datos de entrenamiento
vectorizer = CountVectorizer()
vectorizer.fit(sentences_train)

X_train = vectorizer.transform(sentences_train)
X_test  = vectorizer.transform(sentences_test)

In [7]:
from sklearn.linear_model import LogisticRegression
#Creamos y entrenamos el modelo
classifier = LogisticRegression()
classifier.fit(X_train, y_train)
#Testeamos el modelo con los datos de testeo
score = classifier.score(X_test, y_test)

print("Precisión:", score)
classifier.predict(X_test)

Precisión: 0.788


array([0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0,
       0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1,
       0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0,
       0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0,
       1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1,
       1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0,
       0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1,
       1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1,
       0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0,
       1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0,
       0, 0, 1, 1, 0, 0, 1, 1])

In [8]:
#Creamos un clasificador con todos los datos de todas las clases.
#Esta vez utilizamos la fuente en lugar de la etiqueta, ya que las etiquetas no nos sirven en este caso.
#Solo queremos conocer la fuente de los comentarios
source = df['source'].values
sentences = df['sentence'].values

#Creamos los datos de entrenamiento
sentences_train, sentences_test, y_train, y_test = train_test_split(
    sentences, source, test_size=0.25, random_state=1000)

#Los vectorizamos
vectorizer = CountVectorizer()
vectorizer.fit(sentences_train)
X_train = vectorizer.transform(sentences_train)
X_test  = vectorizer.transform(sentences_test)

#Creamos ye entrenamos el modelo
classifier = LogisticRegression()
classifier.fit(X_train, y_train)

#Testeamos el modelo
score = classifier.score(X_test, y_test)
print('Precisión para datos {}: {:.4f}'.format(source, score))

Precisión para datos ['yelp' 'yelp' 'yelp' ... 'amazon' 'amazon' 'amazon']: 0.8173


In [9]:
#Comentario de Amazon
s=['Creo que Tom Hanks hace un papel espectacular']

In [10]:
#Probamos el clasificador para el comentario definido en la celda de arriba
x = vectorizer.transform(s)
x=classifier.predict(x)
dataframe=pd.DataFrame(x, columns=['Predictions'])
dataframe

,Predictions
0,imdb


In [11]:
#Comentario de Amazon
s=['Este producto es ideal']
x = vectorizer.transform(s)
x=classifier.predict(x)
dataframe=pd.DataFrame(x, columns=['Predictions'])
dataframe

,Predictions
0,amazon
